In [1]:
# !pip install pandas pymysql

# !pip install gensim

# !pip install spacy

# %pip install pyarrow

# %pip install cryptography


# !pip install pyspellchecker

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import re,string

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from spellchecker import SpellChecker
from nltk.corpus import wordnet

# Download necessary NLTK data
import nltk
nltk.download('punkt')
nltk.download('wordnet')


from flask import Flask, request, jsonify
from flask_cors import CORS

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     UNEXPECTED_EOF_WHILE_READING] EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     UNEXPECTED_EOF_WHILE_READING] EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:1006)>


In [3]:
import pymysql

In [4]:
mysql_host = 'localhost'
mysql_port = 3307,
mysql_user = 'root'
mysql_password = 'root'
mysql_db = 'sneakfit'

In [5]:
conn = pymysql.connect(
     host=mysql_host,
    port=3307,
     user=mysql_user,
     password=mysql_password,
     db=mysql_db
)

In [6]:
cursor = conn.cursor()

In [7]:
query = """
    SELECT
        p.product_id,
        p.product_name,
        p.product_description,
        p.price,
        p.discount_price,
        p.product_color,
        p.product_image,
        p.product_category,
        p.product_subcategory,
        p.product_brand,
        ps.size_id
    FROM
        product p
            JOIN
        product_sizes ps_rel ON p.product_id = ps_rel.product_id
            JOIN
        product_size ps ON ps_rel.size_id = ps.size_id
    ORDER BY
        p.product_id;
    """

In [8]:
cursor.execute(query)

130

In [9]:
data = cursor.fetchall()

In [10]:
for table in data:
    print(table)

('PO01', "Velocity NITRO™ 3 Women's Running Shoes", "The PUMA Velocity NITRO™ 3 – with this shoe, we're taking industry-best and bringing it to you. These kicks are all about speed and comfort thanks to advanced NITRO™ technology, offering superior responsiveness and cushioning in a lightweight package that feels like you're running on clouds. Plus, with the heel spoiler, you not only get a sleek look but you'll also benefit from enhanced stability for confident strides. Get ready to up your running game with the Velocity NITRO™ 3. It's not just a shoe; it's a statement", Decimal('135.00'), Decimal('33.75'), 'Neon Citrus-PUMA Silver-Silver Mist', 'Velocity-NITRO™-3-Women_s-Running-Shoes.png', 'XO3', 'SUB01', 'BO3', 'SO4')
('PO01', "Velocity NITRO™ 3 Women's Running Shoes", "The PUMA Velocity NITRO™ 3 – with this shoe, we're taking industry-best and bringing it to you. These kicks are all about speed and comfort thanks to advanced NITRO™ technology, offering superior responsiveness and 

In [11]:
col=['productId','productName','productDescription','price','discountPrice','productColor', 'productImage', 
     'productCategory', 'productSubCategory' , 'productBrand','productSize']

In [12]:
 query = """
    WITH size_aggregation AS (
        SELECT
            ps_rel.product_id,
            GROUP_CONCAT(ps.size_name ORDER BY ps.size_id SEPARATOR ', ') AS product_sizes
        FROM
            product_sizes ps_rel
            JOIN product_size ps ON ps_rel.size_id = ps.size_id
        GROUP BY
            ps_rel.product_id
    )
    
    SELECT
        p.product_id AS productId,
        p.product_name AS productName,
        p.product_description AS productDescription,
        p.price AS price,
        p.discount_price AS discountPrice,
        p.product_color AS productColor,
        p.product_image AS productImage,
        c.category_name AS productCategory,
        t.sub_category_name AS productSubCategory,
        b.brand_name AS productBrand,
        sa.product_sizes AS productSize
    FROM
        product p
        JOIN product_brand b ON p.product_brand = b.brand_id
        JOIN product_category c ON p.product_category = c.category_id
        JOIN product_subcategory t ON p.product_subcategory = t.sub_category_id
        LEFT JOIN size_aggregation sa ON p.product_id = sa.product_id
    ORDER BY
        p.product_id;
    """

In [13]:
cursor.execute(query)

47

In [14]:
data = cursor.fetchall()

In [15]:
df = pd.DataFrame(data, columns = col)

In [16]:
df.head()

,productId,productName,productDescription,price,discountPrice,productColor,productImage,productCategory,productSubCategory,productBrand,productSize
0,PO01,Velocity NITRO™ 3 Women's Running Shoes,"The PUMA Velocity NITRO™ 3 – with this shoe, w...",135.00,33.75,Neon Citrus-PUMA Silver-Silver Mist,Velocity-NITRO™-3-Women_s-Running-Shoes.png,Women,Running,Puma,"M, L, XL"
1,PO02,LOVE MARATHON ForeverRun NITRO™ Men's Running ...,Run proud with the LOVE MARATHON ForeverRun Ni...,150.00,None,PUMA Black-PUMA White,LOVE-MARATHON-ForeverRun-NITRO™-Men_s-Running-...,Men,Running,Puma,"S, M, L"
2,PO03,Nike Pegasus 41,Responsive cushioning in the Pegasus provides ...,140.00,None,Pink,pegasus-41-womens-road-running-shoes-tSbZGh.png,Women,Running,Nike,"S, M, L"
3,PO04,Nike Pegasus 41,Responsive cushioning in the Pegasus provides ...,140.00,None,"Night Sky Blue, Dusty Rose Pink, Forest Moss G...",pegasus-41-mens-road-running-shoes-7S90QB.png,Men,Running,Nike,"M, L, XL"
4,PO05,GALAXY 6 SHOES,"It was true then, and it's true now. adidas de...",60.00,29.00,"Flash Aqua, Cloud White, Magic Grey Met",Galaxy_6_Shoes_Turquoise_IE1992_01_standard.png,Women,Running,Adidas,"S, M, L"


In [17]:
slected_col = ['productId', 'price', 'productBrand','productCategory', 'productColor', 'productSize', 'productSubCategory']

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   productId           47 non-null     object
 1   productName         47 non-null     object
 2   productDescription  47 non-null     object
 3   price               47 non-null     object
 4   discountPrice       16 non-null     object
 5   productColor        47 non-null     object
 6   productImage        47 non-null     object
 7   productCategory     47 non-null     object
 8   productSubCategory  47 non-null     object
 9   productBrand        47 non-null     object
 10  productSize         47 non-null     object
dtypes: object(11)
memory usage: 4.2+ KB


In [19]:
df.describe()

,productId,productName,productDescription,price,discountPrice,productColor,productImage,productCategory,productSubCategory,productBrand,productSize
count,47,47,47,47,16,47,47,47,47,47,47
unique,47,45,44,27,15,47,47,3,10,4,7
top,PO01,Nike Pegasus 41,Responsive cushioning in the Pegasus provides ...,130.00,900.00,Neon Citrus-PUMA Silver-Silver Mist,Velocity-NITRO™-3-Women_s-Running-Shoes.png,Women,Classic,Nike,"M, L, XL"
freq,1,2,2,4,2,1,1,19,11,16,20


In [20]:
#!pip install flask-cors

In [21]:
encoder = OneHotEncoder()

In [22]:
encoded_features = encoder.fit_transform(df[['productBrand', 'productColor', 'productSize','productCategory', 'productSubCategory']])
    

In [23]:
cosine_sim = cosine_similarity(encoded_features)

In [24]:
def get_similar_products(product_id, cosine_sim, df):
    idx = df.index[df['productId'] == product_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Get top 5 similar products
    product_indices = [i[0] for i in sim_scores]
    return df.iloc[product_indices][['productId','productImage', 'productName','price','discountPrice','productBrand']]

In [25]:
similar_products = get_similar_products("PO01", cosine_sim, df)
print(similar_products)

   productId                                       productImage  \
26      PO27                 Mayze-Leather-Women's-Sneakers.png   
1       PO02  LOVE-MARATHON-ForeverRun-NITRO™-Men_s-Running-...   
2       PO03    pegasus-41-womens-road-running-shoes-tSbZGh.png   
3       PO04      pegasus-41-mens-road-running-shoes-7S90QB.png   
4       PO05    Galaxy_6_Shoes_Turquoise_IE1992_01_standard.png   

                                          productName   price discountPrice  \
26                     Mayze Leather Women's Sneakers   90.00          None   
1   LOVE MARATHON ForeverRun NITRO™ Men's Running ...  150.00          None   
2                                     Nike Pegasus 41  140.00          None   
3                                     Nike Pegasus 41  140.00          None   
4                                      GALAXY 6 SHOES   60.00         29.00   

   productBrand  
26         Puma  
1          Puma  
2          Nike  
3          Nike  
4        Adidas  


In [26]:

# Define the selected columns
selected_col =  ['productName', 'productDescription','productBrand', 'productColor', 'productSubCategory', 'productCategory']


# Concatenate the selected columns into a single string for each row
df['combined_text'] = df[selected_col].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
df['combined_text'] = df['combined_text'].str.lower()

In [27]:
# selected_col_tag = ['productBrand', 'productColor', 'productSubCategory', 'productCategory']
# df['tag'] = df[selected_col_tag].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
# df['tag'] = df['tag'].str.lower()




In [28]:
encoder = OneHotEncoder()
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])
cosine_similarities = cosine_similarity(tfidf_matrix)

In [29]:
contents = df['combined_text'].tolist()
contents_encoded = encoder.fit_transform([[content] for content in contents])

# Print one-hot encoded matrix
print("One-Hot Encoded Categories:")
print(contents_encoded.toarray())

One-Hot Encoded Categories:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [30]:
# # Function to recommend products based on query
# def recommend_products(query, vectorizer, tfidf_matrix, df):
#     # Transform query into TF-IDF vector
#     query_vec = vectorizer.transform([query.lower()])
    
#     # Calculate cosine similarity between query vector and all product vectors
#     cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    
#     # Get indices of products sorted by similarity scores
#     sim_scores_indices = list(enumerate(cosine_similarities))
#     sim_scores_indices = sorted(sim_scores_indices, key=lambda x: x[1], reverse=True)
    
#     # Extract indices of top 10 similar products
#     top_indices = [i[0] for i in sim_scores_indices[:10]]
#     top_scores = [i[1] for i in sim_scores_indices[:10]]
    
#     # If no product matches the query, return a message
#     if top_scores[0] == 0:
#         return "No product found"
    
#     # Return recommended product names with similarity scores
#     recommendations = df.iloc[top_indices][['productName', 'combined_text', 'tags']].copy()
#     recommendations['score'] = top_scores
    
#     return recommendations

In [31]:
def get_similar_words(word):
    similar_words = set()

    # Use WordNet to find synonyms and related words
    synsets = wordnet.synsets(word)

    # Collect synonyms, hypernyms, hyponyms, and related forms
    for synset in synsets:
        for lemma in synset.lemmas():
            similar_words.add(lemma.name())
        
        for hypernym in synset.hypernyms():
            for lemma in hypernym.lemmas():
                similar_words.add(lemma.name())
        
        for hyponym in synset.hyponyms():
            for lemma in hyponym.lemmas():
                similar_words.add(lemma.name())
        
        for derivationally_related_form in synset.lemmas()[0].derivationally_related_forms():
            similar_words.add(derivationally_related_form.name())

    # Exclude the original word from the result
    similar_words.discard(word)

    return list(similar_words)[:4]

# Debugging the get_similar_words function for "trekking"
similar_words = get_similar_words("sneaker")

# Print the results
print("Similar words:")
print(", ".join(similar_words))  # Print top 10 similar words


Similar words:
plimsoll, shoe, betrayer, sneak


In [32]:
# Initialize the SpellChecker
spell = SpellChecker()

# Add custom words to the spell checker
custom_words = ['nike','adidas','puma']
spell.word_frequency.load_words(custom_words)

def correct_spell(query):
    corrected_query = []
    words = query.split()

    for word in words:
        corrected_query.append(spell.correction(word))

    return " ".join(corrected_query)

# Test the correct_spell function
print(correct_spell("wlaks")) 

walks


In [33]:
def recommend_products(query, cosine_similarities, df):

    query = correct_spell(query)
    
    search_query = []
    
    similar_words = get_similar_words(query)
    print(similar_words)
    
    for index, row in df.iterrows():
        content = row['combined_text']
        if query in content:
            search_query.append(index)
    
   
    results = []
    if search_query:
       
        for index in search_query:
            similar_items = list(enumerate(cosine_similarities[df.index.get_loc(index)]))
            sorted_similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[:10]

            for item in sorted_similar_items:
                product_id, score = item

                # Check if the product_id already exists in results
                found = False
                for i, (existing_product_id, existing_score) in enumerate(results):
                    if existing_product_id == product_id:
                        found = True
                        # Update the score if the current score is higher
                        if score > existing_score:
                            results[i] = (product_id, score)
                        break

                # If not found, add the item to the results
                if not found:
                    results.append((product_id, score))
    
        return sorted(results, key=lambda x: x[1], reverse=True)
    
    # If no exact matches are found, look for partial matches and similar words
    else:
        for index, row in df.iterrows():
            content = row['combined_text']
            if any(word in content for word in query.split() + similar_words):
                similar_items = list(enumerate(cosine_similarities[df.index.get_loc(index)]))
                sorted_items = sorted(similar_items, key=lambda x: x[1], reverse=True)
                results.extend(sorted_items)
                
        result = list(set(results))
        return sorted(result, key=lambda x: x[1], reverse=True)[:15]

In [34]:
# Test the recommend_products function
query = 'trek'
recommended_products = recommend_products(query, cosine_similarities, df)

# Print the results
print("Recommended Products:")
for index, score in recommended_products:
    print(f"Product ID: {df.iloc[index]['productId']}, Score: {score:.2f},  Category: {df.iloc[index]['productSubCategory']}")

['trip', 'trekker', 'journeying', 'travel']
Recommended Products:
Product ID: PO17, Score: 1.00,  Category: Hiking
Product ID: PO13, Score: 1.00,  Category: Hiking
Product ID: PO18, Score: 0.92,  Category: Hiking
Product ID: PO14, Score: 0.26,  Category: Hiking
Product ID: PO01, Score: 0.17,  Category: Running
Product ID: PO19, Score: 0.16,  Category: Running
Product ID: PO02, Score: 0.13,  Category: Running
Product ID: PO11, Score: 0.12,  Category: Gym and Training
Product ID: PO32, Score: 0.11,  Category: Football
Product ID: PO38, Score: 0.10,  Category: Cricket
Product ID: PO07, Score: 0.10,  Category: Gym and Training
Product ID: PO06, Score: 0.10,  Category: Running
Product ID: PO15, Score: 0.09,  Category: Hiking
Product ID: PO16, Score: 0.09,  Category: Hiking
Product ID: PO20, Score: 0.07,  Category: Hiking


In [35]:
app = Flask(__name__)
CORS(app, resources={r"/api/*": {"origins": "http://localhost:5173"}})

In [36]:
# Define the search products route
@app.route('/api/products/search', methods=['GET'])
def search_products():
     query = request.args.get('query')
     # Call find_similar_products function
     recommended_products = recommend_products(query, cosine_similarities, df)
        
     # Convert the results to a DataFrame to JSON
     recommended_products_df = df.iloc[[idx for idx, _ in recommended_products]]
     recommended_products_json = recommended_products_df.to_dict(orient='records')
        
     return jsonify(recommended_products_json)
   
@app.route('/api/product/<product_id>', methods=['GET'])
def api_similar_products(product_id):
    similar_products = get_similar_products(product_id, cosine_sim, df)
    similar_products_json = similar_products.to_dict(orient='records')
    return jsonify(similar_products_json)


In [37]:
# Run the Flask application in the background
if __name__ == '__main__':
    app.run('localhost',port=3000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:3000
Press CTRL+C to quit
127.0.0.1 - - [29/Jul/2024 07:28:21] "GET /api/products/search?query=walk HTTP/1.1" 200 -


['last_mile', 'flounce', 'skywalk', 'parade']


127.0.0.1 - - [29/Jul/2024 07:28:21] "GET /api/products/search?query=walk HTTP/1.1" 200 -


['last_mile', 'flounce', 'skywalk', 'parade']


127.0.0.1 - - [29/Jul/2024 07:28:24] "GET /api/product/PO05 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:28:25] "GET /api/product/PO05 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:31:25] "GET /api/product/PO01 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:31:25] "GET /api/product/PO01 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:47:48] "GET /api/product/PO02 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:47:48] "GET /api/product/PO02 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:50:21] "GET /api/product/PO35 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:50:21] "GET /api/product/PO35 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:50:30] "GET /api/product/PO40 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:50:32] "GET /api/product/PO25 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:50:33] "GET /api/product/PO04 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:50:34] "GET /api/product/PO26 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 07:50:35] "GET /api/product/PO04 HTTP/1.1" 200 -
127.0.0.1 - 

In [ ]:
# # def recommend_products(query, cosine_similarities, df):
# #     query = correct_spell(query)
#       for index in search_query:
       
#             similar_items = list(enumerate(cosine_similarities[df.index.get_loc(index)]))
#             sorted_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[:10]
#             results.extend(sorted_items)
        
#         result = list(set(results))
#         return sorted(result, key=lambda x: x[1], reverse=True)
    
#     # If no exact matches are found, look for partial matches and similar words
# #     search_query = []
# #     similar_words = get_similar_words(query)
# #     print(similar_words)
    
# #     for index, row in df.iterrows():
# #         content = row['combined_text']
# #         if query in content:
# #             search_query.append(index)
    
# #     results = []
# #     unique_indices = set()

# #     if search_query:
# #         for index in search_query:
# #             similar_items = list(enumerate(cosine_similarities[df.index.get_loc(index)]))
# #             sorted_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[:10]
# #             for item in sorted_items:
# #                 if item[0] not in unique_indices:
# #                     unique_indices.add(item[0])
# #                     results.append(item)
        
# #         return sorted(results, key=lambda x: x[1], reverse=True)
    
# #     # If no exact matches are found, look for partial matches and similar words
# #     else:
# #         for index, row in df.iterrows():
# #             content = row['combined_text']
# #             if any(word in content for word in query.split() + similar_words):
# #                 similar_items = list(enumerate(cosine_similarities[df.index.get_loc(index)]))
# #                 sorted_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[:5]
# #                 for item in sorted_items:
# #                     if item[0] not in unique_indices:
# #                         unique_indices.add(item[0])
# #                         results.append(item)

# #         return sorted(results, key=lambda x: x[1], reverse=True)


In [ ]:
# def find_similar_products(query, vectorizer, tfidf_matrix, df, top_n=10):
#     ps = PorterStemmer()

#     query = clean_text(query)
#     query_vector = vectorizer.transform([query])
#     similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
#     top_indices = similarity_scores.argsort()[::-1]
#     # top_scores = similarity_scores[top_indices]

#     if max(top_scores) >= threshold:
#         recommended_products = df.iloc[top_indices[:top_n]]
#         return recommended_products

#     # If no products meet the similarity threshold, find products with similar terms
#     query_terms = word_tokenize(query)
#     stemmed_query_terms = [ps.stem(term) for term in query_terms]

#     # Function to get synonyms from WordNet
#     def get_synonyms(word):
#         synonyms = set()
#         for syn in wordnet.synsets(word):
#             for lemma in syn.lemmas():
#                 synonyms.add(lemma.name().lower())
#         return synonyms

#     # Extend query terms with synonyms
#     synonym_query_terms = set(stemmed_query_terms)
#     for term in query_terms:
#         term_synonyms = get_synonyms(term)
#         synonym_query_terms.update(ps.stem(syn) for syn in term_synonyms)

#     def contains_stemmed_terms(text):
#         text_terms = word_tokenize(text.lower())
#         stemmed_text_terms = [ps.stem(term) for term in text_terms]
#         return any(stemmed_term in stemmed_text_terms for stemmed_term in synonym_query_terms)

#     mask = df['combined_text'].apply(contains_stemmed_terms)
#     similar_term_products = df[mask]

#     if similar_term_products.empty:
#         return pd.DataFrame(columns=df.columns)

#     return similar_term_products.head(top_n)


In [ ]:

# def recommend_products(search_query, df, tfidf_vectorizer, tfidf_matrix, limit=10):
#     ps = PorterStemmer()

#     query = clean_text(search_query)
#     # Vectorize search query
#     query_vector = tfidf_vectorizer.transform([query.lower()])

#     # Calculate cosine similarities
#     similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

#     # Get indices of top similar products
#     top_indices = similarities.argsort()[::-1][:limit]
#     top_scores = similarities[top_indices]

#     if recommend_products:
#         recommended_products = df.iloc[top_indices[:limit]]
#     else:
#         # If no products meet the similarity threshold, find products with similar terms
#         query_terms = word_tokenize(query)
#         stemmed_query_terms = [ps.stem(term) for term in query_terms]

#         # Function to get synonyms from WordNet
#         def get_synonyms(word):
#             synonyms = set()
#             for syn in wordnet.synsets(word):
#                 for lemma in syn.lemmas():
#                     synonyms.add(lemma.name().lower())
#             return synonyms

#         # Extend query terms with synonyms
#         synonym_query_terms = set(stemmed_query_terms)
#         for term in query_terms:
#             term_synonyms = get_synonyms(term)
#             synonym_query_terms.update(ps.stem(syn) for syn in term_synonyms)

#         def contains_stemmed_terms(text):
#             text_terms = word_tokenize(text.lower())
#             stemmed_text_terms = [ps.stem(term) for term in text_terms]
#             return any(stemmed_term in stemmed_text_terms for stemmed_term in synonym_query_terms)

#         mask = df['combined_text'].apply(contains_stemmed_terms)
#         similar_term_products = df[mask]

#         if similar_term_products.empty:
#             return pd.DataFrame(columns=df.columns)

#         recommended_products = similar_term_products.head(limit)

#     return recommended_products


In [ ]:
# # Function to find similar products based on a query using Word2Vec embeddings
# def find_similar_products(query, model, df, threshold=0.2, top_n=10):
#     query_tokens = clean_text(query)
#     if not query_tokens:
#         return pd.DataFrame(columns=df.columns)  # Return empty DataFrame if query has no tokens

#     # Compute average vector for query tokens present in the model's vocabulary
#     query_vector = np.mean([model[token] for token in query_tokens if token in model], axis=0)

#     similarity_scores = []
#     for idx, row in df.iterrows():
#         text_tokens = clean_text(row['combined_text'])
#         if not text_tokens:
#             similarity_scores.append(0.0)  # Append zero similarity if text has no tokens
#             continue
        
#         # Compute average vector for text tokens present in the model's vocabulary
#         text_vector = np.mean([model[token] for token in text_tokens if token in model], axis=0)
        
#         if text_vector is None or text_vector.size == 0:
#             similarity_scores.append(0.0)  # Append zero similarity if no tokens are in vocabulary
#             continue
        
#         # Reshape vectors to ensure they are 2D arrays
#         query_vector_reshaped = query_vector.reshape(1, -1)
#         text_vector_reshaped = text_vector.reshape(1, -1)
        
#         # Calculate cosine similarity
#         similarity = cosine_similarity(query_vector_reshaped, text_vector_reshaped)[0][0]
#         similarity_scores.append(similarity)

#     df['similarity_score'] = similarity_scores
#     recommended_products = df[df['similarity_score'] >= threshold].nlargest(top_n, 'similarity_score')

#     if recommended_products.empty:
#         return pd.DataFrame(columns=df.columns)  # Return empty DataFrame if no products meet threshold

#     return recommended_products[['productId', 'productName', 'productImage', 'similarity_score']]

In [ ]:


# # TF-IDF Vectorization
# tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_text'])

In [ ]:
# def recommend_products(search_query, df, tfidf_vectorizer, tfidf_matrix, limit=10):
#     # Vectorize search query
#     query_vector = tfidf_vectorizer.transform([search_query.lower()])

#     # Calculate cosine similarities
#     similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

#     # Get indices of top similar products
#     top_indices = similarities.argsort()[::-1][:limit]

#     # Retrieve recommended products with details
#     recommended_products = df.iloc[top_indices]

#     return recommended_products


In [ ]:
# search_query = "gold"
# recommended_products = recommend_products(search_query, df, tfidf_vectorizer, tfidf_matrix)

# # Display recommended products
# for index, product in recommended_products.iterrows():
#     print(f"Product ID: {index}")  # Assuming index is used as product ID
#     print(f"Product Name: {product['productName']}")
#     print(f"Product Category: {product['productSubCategory']}")
#     print(f"Product Brand: {product['productBrand']}")
#     print()

In [ ]:
# titles=df[['productName','tags']]
# df.head()